In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/UCR/Deep Learning/project/
!ls

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/UCR/Deep Learning/project
adaboost       DenseNet.ipynb  plot1.png  test.obj   validation.obj
covid19.model  mdl_wts.hdf5    plot2.png  train.obj


In [ ]:
import pickle
dataset_path = "./"
file = open(dataset_path + 'train.obj','rb')
object_file = pickle.load(file)
trainX = object_file['images']
trainY = object_file['labels']

file = open(dataset_path + 'validation.obj','rb')
object_file = pickle.load(file)
validationX = object_file['images']
validationY = object_file['labels']


file = open(dataset_path + 'test.obj','rb')
object_file = pickle.load(file)
testX = object_file['images']
testY = object_file['labels']

In [ ]:
import numpy as np
train_labels = np.where(trainY == 1)[1]
train_labels = np.reshape(train_labels, (-1, 1))

validation_labels = np.where(validationY == 1)[1]
validation_labels = np.reshape(validation_labels, (-1, 1))

test_labels = np.where(testY == 1)[1]
test_labels = np.reshape(test_labels, (-1, 1))

In [ ]:
indices = np.where(train_labels == 0)[0]
trainX = np.concatenate((trainX, validationX),axis=0)
train_labels = np.concatenate((train_labels, validation_labels),axis=0)

for i in range(3):
    temp = trainX[indices]
    trainX = np.concatenate((trainX, temp),axis=0)

    temp = train_labels[indices]
    train_labels = np.concatenate((train_labels, temp),axis=0)
print(trainX.shape, train_labels.shape)

(2748, 224, 224, 3) (2748, 1)


In [ ]:
!pip install tensorflow-gpu==1.15
import tensorflow as tf
tf.test.gpu_device_name()
from keras import backend as K
print(K.tensorflow_backend._get_available_gpus())
import keras

     |████████████████████████████████| 411.5MB 45kB/s 
     |████████████████████████████████| 512kB 36.2MB/s 
     |████████████████████████████████| 3.8MB 38.2MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=4825b0642ec544ede3d0871c36bbc58741c6801fc0fc0ec8c3ff292fb7ab7f9f
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 1.15.0 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 1.15.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.

Using TensorFlow backend.


In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import Model

base = './adaboost/'
end = '/covid19.model'
model_paths = ['VGG', 'ResNet50', 'DenseNet']
for j in range(len(model_paths)):
    path = base+model_paths[j]+end
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path)
    input_layers = []
    for i in range(len(model.layers)-2):
        input_layers.append(model.layers[i])
    feature_extractor = Model(inputs=model.inputs, outputs=model.layers[-2].output)

    output = feature_extractor.predict(trainX, batch_size=16)
    if j == 0:
        train_features = output
    else:
        train_features = np.concatenate((train_features, output),axis=1)

    output = feature_extractor.predict(testX, batch_size=16)
    if j == 0:
        test_features = output
    else:
        test_features = np.concatenate((test_features, output),axis=1)
    print(test_features.shape, type(test_features))

    categories = ['covid', 'normal', 'ph']
    print("[INFO] evaluating network...")
    predIdxs = model.predict(testX, batch_size=16)
    predIdxs = np.argmax(predIdxs, axis=1)
    print(classification_report(testY.argmax(axis=1), predIdxs, target_names=categories))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(580, 512) <class 'numpy.ndarray'>
[INFO] evaluating network...
              precision    recall  f1-score   support

       covid       0.91      0.95      0.93        43
      normal       0.91      0.97      0.94       268
          ph       0.96      0.90      0.93       269

    accuracy                           0.93       580
   macro avg       0.93      0.94      0.93       580
weighted avg       0.94      0.93      0.93       580

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
(580, 2560) <class 'numpy.ndarray'>
[INFO] evaluating network...
              precision    recall  f1-score   support



In [ ]:
print(train_features.shape, type(train_features))
print(train_labels.shape, type(train_labels))

(2748, 96640) <class 'numpy.ndarray'>
(2748, 1) <class 'numpy.ndarray'>


In [ ]:
X = train_features[:,0:6000]

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=150, random_state=0)
clf.fit(X, train_labels)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=300, random_state=0)

In [ ]:
predictions = clf.predict(test_features[:,0:6000])
predictions = np.reshape(predictions, (-1, 1))

In [ ]:
print(predictions.shape)
print(test_labels.shape)

(580, 1)
(580, 1)


In [ ]:
from sklearn.metrics import accuracy_score,  confusion_matrix
print(accuracy_score(test_labels, predictions))
print(confusion_matrix(test_labels, predictions))

0.9568965517241379
[[ 41   0   2]
 [  0 258  10]
 [  0  13 256]]


In [ ]:
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)
predictions = np.reshape(predictions, (-1, 1))
print(accuracy_score(test_labels, predictions))
print(confusion_matrix(test_labels, predictions))